<a href="https://colab.research.google.com/github/richeung1/Fake-News-Detector/blob/main/Fake_News_Detector_26Apr2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Fake News Detector***



In [ ]:
# Importing Python Packages

# Pandas and Numpy for working with dataframe/arrays
import pandas as pd
import numpy as np

# NLTK for tokenization (splitting sentences into single words)
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# TfidfVectorizer for changing words into numerical data (necessary for machine learning models)
from sklearn.feature_extraction.text import TfidfVectorizer

# Train Test Split to split up the data into a training set (data that model uses for training) and a test set (unseen data that model uses for predictions)
from sklearn.model_selection import train_test_split

# 3 different machine learning models (each model uses different math/statistics): Logistic Regression, K-Nearest Neighbors, and Random Forest
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Accuracy score as a metric to compare machine learning model's predictions from the unseen test set vs the unseen test set's actual data
from sklearn.metrics import accuracy_score

# GridSearchCV to finetune the model's hyperparamaters (each model comes with extra settings that can be adjusted to achieve better accuracy)
from sklearn.model_selection import GridSearchCV


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Loading data: news_articles.csv. Then doing exploratory data analysis (EDA) and any required data clean up.**

In [ ]:
# Loading news_articles.csv into a pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/news_articles.csv')

In [ ]:
df.head(5)

,author,published,title,text,language,site_url,main_img_url,type,label,title_without_stopwords,text_without_stopwords,hasImage
0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,muslims busted they stole millions in govt ben...,print they should pay all the back all the mon...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,muslims busted stole millions govt benefits,print pay back money plus interest entire fami...,1.0
1,reasoning with facts,2016-10-29T08:47:11.259+03:00,re why did attorney general loretta lynch plea...,why did attorney general loretta lynch plead t...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,attorney general loretta lynch plead fifth,attorney general loretta lynch plead fifth bar...,1.0
2,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,breaking weiner cooperating with fbi on hillar...,red state \nfox news sunday reported this mor...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,breaking weiner cooperating fbi hillary email ...,red state fox news sunday reported morning ant...,1.0
3,Fed Up,2016-11-01T05:22:00.000+02:00,pin drop speech by father of daughter kidnappe...,email kayla mueller was a prisoner and torture...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,pin drop speech father daughter kidnapped kill...,email kayla mueller prisoner tortured isis cha...,1.0
4,Fed Up,2016-11-01T21:56:00.000+02:00,fantastic trumps point plan to reform healthc...,email healthcare reform to make america great ...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,fantastic trumps point plan reform healthcare ...,email healthcare reform make america great sin...,1.0


In [ ]:
# Shows we have 2,096 rows by 12 columns
df.shape

(2096, 12)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2096 entries, 0 to 2095
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   author                   2096 non-null   object 
 1   published                2096 non-null   object 
 2   title                    2096 non-null   object 
 3   text                     2050 non-null   object 
 4   language                 2095 non-null   object 
 5   site_url                 2095 non-null   object 
 6   main_img_url             2095 non-null   object 
 7   type                     2095 non-null   object 
 8   label                    2095 non-null   object 
 9   title_without_stopwords  2094 non-null   object 
 10  text_without_stopwords   2046 non-null   object 
 11  hasImage                 2095 non-null   float64
dtypes: float64(1), object(11)
memory usage: 196.6+ KB


In [ ]:
# Shows we have N/A or NaN values in various columns such as text, label, etc. Machine Learning models have issues with N/A or NaN values since these are missing values.
df.isnull().sum()

author                      0
published                   0
title                       0
text                       46
language                    1
site_url                    1
main_img_url                1
type                        1
label                       1
title_without_stopwords     2
text_without_stopwords     50
hasImage                    1
dtype: int64

In [ ]:
# Creating a new dataframe with only the columns that I want: text_without_stopwords (text of the news articles without filler/useless words such as "like", "and", etc.). In simple terms this column is the X variable.
new_df = pd.DataFrame(df['text_without_stopwords'])

In [ ]:
# Second column of the new dataframe is: labels because ultimately this is what we want to predict: Real vs. Fake. This column is basicaly the Y variable.
new_df['labels'] = df['label']

In [ ]:
# Deciding to drop the N/A and NaN values from the new dataframe so all rows/columns no longer have missing values.
new_df = new_df.dropna()

In [ ]:
new_df.head(5)

,text_without_stopwords,labels
0,print pay back money plus interest entire fami...,Real
1,attorney general loretta lynch plead fifth bar...,Real
2,red state fox news sunday reported morning ant...,Real
3,email kayla mueller prisoner tortured isis cha...,Real
4,email healthcare reform make america great sin...,Real


In [ ]:
# Changing the values of the labels column into numbers. Machine Learning models often work best with numerical data.
new_df['labels'] = new_df['labels'].map({'Real': 1, 'Fake': 0})

In [ ]:
new_df.head(5)

,text_without_stopwords,labels
0,print pay back money plus interest entire fami...,1
1,attorney general loretta lynch plead fifth bar...,1
2,red state fox news sunday reported morning ant...,1
3,email kayla mueller prisoner tortured isis cha...,1
4,email healthcare reform make america great sin...,1


In [ ]:
new_df.shape

(2046, 2)

**Word Tokenization: turning each news article's sentences into individual words. Individual words are the building blocks of language so this transformation is necessary for natural language processing (NLP) tasks which is what we are doing.**

In [ ]:
new_df['text_without_stopwords'] = new_df['text_without_stopwords'].apply(lambda x: word_tokenize(str(x)))

In [ ]:
# News articles in text_without_stopwords colulmn has been transformed into individual words separted by a comma.
new_df.head(5)

,text_without_stopwords,labels
0,"[print, pay, back, money, plus, interest, enti...",1
1,"[attorney, general, loretta, lynch, plead, fif...",1
2,"[red, state, fox, news, sunday, reported, morn...",1
3,"[email, kayla, mueller, prisoner, tortured, is...",1
4,"[email, healthcare, reform, make, america, gre...",1


**Vectorizing: transforming tokens (individual words) into numerical data. This is necessary since machine learning models use math/statistics to find patterns in data.**

In [ ]:
Vectorizer = TfidfVectorizer()

In [ ]:
X = Vectorizer.fit_transform(new_df['text_without_stopwords'].astype(str))
y = new_df['labels'].values

**Train Test Split to split up the data into a training set (data that model uses for training) and a test set (unseen data that model uses for predictions).**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.33, 
    shuffle=False
)


**Training Logistic Regression model.**

In [ ]:
# Logistic Regression

clf = LogisticRegression()

In [ ]:
clf.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# Initial accuracy score from Logistic Regression ~29.59%
y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.2958579881656805


**Training K-Nearest Neighbors model.**

In [ ]:
# KNNeighbors

klf = KNeighborsClassifier(n_neighbors = 5)

In [ ]:
klf.fit(X_train, y_train)

KNeighborsClassifier()

In [ ]:
yk_pred = klf.predict(X_test)

In [ ]:
# Initial accuracy score from K-Nearest Neighbors ~33.43%
print(accuracy_score(y_test, yk_pred))

0.3343195266272189


**Training Random Forest model.**

In [ ]:
# Random Forest

rlf = RandomForestClassifier()

In [ ]:
rlf.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
yr_pred = rlf.predict(X_test)

In [ ]:
# Initial accuracy score from Random Forest ~29.59%
print(accuracy_score(y_test, yr_pred))

0.2943786982248521


**Fine Tuning: adjusting machine learning model's settings i.e. paramaters to improve accuracy. GridSearchCV takes in the model and various arguments to perform the trainings and predictions again with the paramaters that you choose.**

In [ ]:
# GridSearchCV on Logistic Regression: using different parameters
penalty = ['l1', 'l2']
C = np.logspace(-3,3,7)
solver = ['newton-cg', 'lbfgs', 'liblinear']

params = dict(penalty = penalty,
              C = C,
              solver = solver)

LR = GridSearchCV(clf, params, cv=5, verbose=1, n_jobs=-1)

In [ ]:
LRGridSearch = LR.fit(X_train, y_train)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
70 fits failed out of a total of 210.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 449, in _check_solver
    % (solver, penalty)
ValueError: Solve

In [ ]:
LRGridSearch = LR.fit(X_train, y_train)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
70 fits failed out of a total of 210.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 449, in _check_solver
    % (solver, penalty)
ValueError: Solve

In [ ]:
# Best parameters for Logistic Regression
LRGridSearch.best_params_

{'C': 1000.0, 'penalty': 'l2', 'solver': 'liblinear'}

In [ ]:
# Shows that if we best parameters for Logistic Regression we would get ~82.63% accuracy vs our initial result of ~29.59%.
LRGridSearch.best_score_

0.8262773722627736

In [ ]:
# GridSearchCV on Random Forest: using different parameters

n_estimators = [50, 75, 100]
max_features = ['auto', 'sqrt', 'log2']
max_depth = [4, 5, 6, 7, 8]
criterion = ['gini', 'entropy']

params = dict(n_estimators = n_estimators,
              max_features = max_features,
              max_depth = max_depth,
              criterion = criterion)

RFGridSearch = GridSearchCV(rlf, params, cv=5, verbose=1, n_jobs=-1)

In [ ]:
RFGridSearch.fit(X_train, y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [50, 75, 100]},
             verbose=1)

In [ ]:
# Best parameters for Random Forest

RFGridSearch.best_params_

{'criterion': 'entropy',
 'max_depth': 8,
 'max_features': 'auto',
 'n_estimators': 75}

In [ ]:
# Shows that if we used best parameters we would get ~79.78% accuracy vs our initial result of ~29.59%.
RFGridSearch.best_score_

0.7978102189781022

In [ ]:
# GridSearchCV on KNNeighbors: using different parameters
n_neighbors = [5,7,10,15]
weights = ['uniform','distance']
leaf_size = [5,10,30,50]
p = [1,2]
n_jobs = [-1]

params = dict(n_neighbors = n_neighbors,
              weights = weights,
              leaf_size = leaf_size,
              p = p,
              n_jobs = n_jobs)

glf = GridSearchCV(klf, params, cv=5, verbose=1, n_jobs=-1)

In [ ]:
KNNGridSearch = glf.fit(X_train, y_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


In [ ]:
# Best parameters for KNNeighbors
KNNGridSearch.best_params_

{'leaf_size': 5, 'n_jobs': -1, 'n_neighbors': 5, 'p': 1, 'weights': 'uniform'}

In [ ]:
# Shows that if we best parameters for KNNeighbors we would get ~80.95% accuracy vs our initial result of ~33.43%.

KNNGridSearch.best_score_

0.8094890510948904